Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [258]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy, copy
from math import ceil


## The *Nim* and *Nimply* classes

In [259]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [260]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [261]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [262]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [263]:
def first_try(state: Nim) -> Nimply:
    """Pick always the first tile of the highest row"""
    move = Nimply(max([i for i,r in enumerate(state.rows) if r > 0]), 1)
    return move

In [264]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [265]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


In [ ]:
NUM_MATCH = 500
NUM_GEN = 10
NUM_IND = 50

In [ ]:
class Nimmer:
    def __init__(self, n: float, p: float) -> None:
        self.n = n
        self.p = p
        self.win_rate = 0

    def __lt__(self, other):
        return self.win_rate < other.win_rate
    
    def __str__(self):
        return "{" + str(self.n) + "," + str(self.p) + "}" + " " + "WIN:" + str(self.win_rate*100)

    def valute(self, wr):
        self.win_rate = wr

In [ ]:
def es_strategy(state: Nim, ind: Nimmer) -> Nimply:
    if random.random() <= ind.p :
        possible_rows = [(r,i) for i,r in enumerate(state.rows) if r > 0]
        possible_rows.sort(reverse=True)
        _ , row = possible_rows.pop(0)
        #print("Pazzo prende max")
    else:
        row = random.choice([i for i, r in enumerate(state.rows) if r > 0])
        #print("Pazzo prende random")
    num_objects = ceil(state.rows[row] * ind.n)
    return Nimply(row,num_objects)


    
    

In [ ]:
def play_matches(ind):
    print(f"TESTING INDIVIDUAL {ind}")
    strategy = (es_strategy, optimal)
    won = [0,0]
    for match in range(NUM_MATCH):
        nim = Nim(5)
        player = 0
        #print(f"Si presenta Pazzo: {ind}")
        while nim:
            if player:
                #print(f"Random si ritrova con: {nim}")
                ply = strategy[player](nim)
                #print(f"Random gioca {ply}")
            else:
                #print(f"Situazione prima di pazzo: {nim}")
                ply = strategy[player](nim,ind)
                #print(f"Pazzo gioca {ply}")
            nim.nimming(ply)
            player = 1 - player
        won[player] += 1
    print(f"Result: {won[0]/NUM_MATCH*100}%")
    return won[0]/NUM_MATCH

In [ ]:
initial_n = 0.5 # How many pieces to take (0 => None 1 => All)
initial_p = 0.5 # Chance to take from row w most pieces, otherwise random

best = Nimmer(initial_n,initial_p)
for gen in range(NUM_GEN):
    new_n = np.random.normal(loc=0,scale=0.1,size=NUM_IND)+best.n
    new_p = np.random.normal(loc=0,scale=0.1,size=NUM_IND)+best.p
    individuals = [Nimmer(n,p) for n,p in zip(new_n,new_p)]
    for ind in individuals:
        ind.valute(play_matches(ind))
    individuals.append(best)
    best = copy(max(individuals, key=lambda i: i.win_rate))
    print(f"Best @ gen. {gen}: {best}")

In [ ]:
VALIDATION_MATCH = 10_000

In [ ]:
hero = copy(best)
print(f"HERO: {hero}")

strategy = (es_strategy, optimal)
won = [0,0]
for match in range(VALIDATION_MATCH):
    nim = Nim(5)
    player = 0
    #print(f"Si presenta Pazzo: {ind}")
    while nim:
        if player:
            #print(f"Random si ritrova con: {nim}")
            ply = strategy[player](nim)
            #print(f"Random gioca {ply}")
        else:
            #print(f"Situazione prima di pazzo: {nim}")
            ply = strategy[player](nim,hero)
            #print(f"Pazzo gioca {ply}")
        nim.nimming(ply)
        player = 1 - player
    won[player] += 1
print(f"TOTAL: {won[0]/VALIDATION_MATCH}")